In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.basemap import Basemap
import numpy as np
import osmnx as ox
import pydeck as pdk
import json
from sklearn import preprocessing

In [ ]:
# Day 3 - Polygons (Vermont Maple Production)
# data sources: https://www.nass.usda.gov/Publications/AgCensus/2022/Full_Report/Volume_1,_Chapter_2_County_Level/Vermont/st50_2_037_038.pdf
#               https://geodata.vermont.gov/datasets/VCGI::vt-data-county-boundaries-1/about

# Read the CSV file
df = pd.read_csv("data/maple_production.csv")

# Create a GeoDataFrame from geojson data 
gdf = gpd.read_file("data/VT_county.geojson")

# Join data
gdf = gdf.merge(df, how='left', left_on='CNTYNAME', right_on='county')

# feature engineering for pydeck viz,  scale production values 0-1
min_max_scaler = preprocessing.MinMaxScaler()
x = gdf["gallons"].values.reshape(-1, 1)
x_scaled = min_max_scaler.fit_transform(x)
gdf["area_norm"] = pd.Series(x_scaled.flatten())

# format data for use in pydeck
json_out = json.loads(gdf.to_json())
# inspect the first authority
json_out["features"][0]["properties"]
r = "250"
g = "(1 - properties.area_norm) * 255"
b = "properties.area_norm * 255"
fill = f"[{r},{g},{b}]"
geojson = pdk.Layer(
        "GeoJsonLayer",
        json_out,
        pickable=True,
        opacity=1,
        stroked=True,
        filled=True,
        extruded=True,
        wireframe=True,
        auto_highlight=True,
        get_elevation="properties.area_norm * 200",
        elevation_scale=100,
        get_fill_color=fill,
    )
tooltip = {"text": "{county}\n{gallons} gallons"}
view_state = pdk.ViewState(
    longitude=-72.7,
    latitude=43.5,
    zoom=7,
    max_zoom=15,
    pitch=50,
    bearing=2,
)
r = pdk.Deck(
    layers=geojson,
    initial_view_state=view_state,
    tooltip=tooltip,
)
r.to_html("maps/3_polygons_map.html")

# Create and save custom colorbar for final map
# Define your RGB color values
colors = [(1, 1, 0), (1, 0.5, 0.5), (1, 0, 1)]  # Red, Green, Blue

# Create a colormap using the defined colors
cmap = mcolors.LinearSegmentedColormap.from_list("my_cmap", colors)
mat = np.random.random((10,10))*1000000
plt.imshow(mat, origin="lower", cmap=cmap, interpolation='nearest')
cbar = plt.colorbar()
cbar.set_label('Gallons Maple Syrup Produced')
# Save the figure
plt.savefig('maps/3_polygons_colorbar.png', dpi=300, bbox_inches='tight')

In [ ]:
# Day 2 - Lines (City street network contrast map)
# data source: openstreetmap

# Select city and crs, note: larger cities take a few minutes
cityname = 'Portland, OR, USA'
crs = 4326

# Select color contrast
color = ['black', 'pink'] # text, background

# Get graph by geocoding
graph = ox.graph_from_place(cityname, network_type="walk")

# Project graph
graph = ox.projection.project_graph(graph, to_crs=crs)

# everything to gdfs
nodes, edges = ox.graph_to_gdfs(graph)

# Setup plot
fig, ax = plt.subplots(figsize=(10,8), dpi=200)
ax.set_axis_off()
ax.set_aspect('equal')
fig.set_facecolor(color[1])

# Plot data
edges.plot(
    ax=ax,
    color=color[0],
    linewidth=0.5
)

# plot city name
plt.annotate(cityname, xy=(0.5, 0), ha='center', xycoords='axes fraction', fontsize=20, color=color[0], weight='bold', family='monospace')

# Tight layout
plt.tight_layout()

# save figure
plt.savefig('maps/2_lines_'+cityname.split()[0][:-1]+'.png', dpi=300, bbox_inches='tight')

In [ ]:
# Day 1 - Points
# data source
# https://www.doogal.co.uk/FootballStadiumsCSV.ashx

# Read the CSV file
df = pd.read_csv("data/stadiums.csv")

# Create a GeoDataFrame from the CSV data
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Draw the map background
fig = plt.figure(figsize=(8, 8), dpi=150)
m = Basemap(projection='lcc', resolution='f',
            width=0.8E6, height=1.1E6, 
            lat_0=54.4, lon_0=-3,)
m.etopo(scale=1, alpha=0.5)

# Scatter stadium data with size reflecting capacity
m.scatter(gdf.Longitude, gdf.Latitude, latlon=True,
          c='goldenrod', s=(30*gdf.Capacity / gdf.Capacity.max())**2,
          alpha=0.7, edgecolors='orangered')

# Make legend with dummy points
for a in [10000, 30000, 50000, 70000]:
    plt.scatter([], [], c='k', alpha=0.7, s=(30*a / gdf.Capacity.max())**2,
                label=str(int(a/1000)) + 'k')
plt.legend(title='Capacity', scatterpoints=1, frameon=True,
           labelspacing=1.2, loc='lower left');
plt.title("UK Football Stadium Capacity", fontdict={'family': 'serif', 'size': 16})

# Add large cities to map
# Add a point with a label
cities = [[-0.1278, 51.5074, '  London'],
          [-2.244644, 53.483959, '  Manchester'],
          [-1.898575, 52.489471, '  Birmingham'],
          [-4.251433, 55.860916, '  Glasgow'],
          [-1.600000, 54.966667, '  Newcastle'],]

for city in cities:
    # Convert lat/lon to map coordinates
    x, y = m(city[0], city[1])
    m.plot(x, y, '.', c='black', markersize=8)
    plt.text(x, y, city[2], fontsize=14, alpha=1, c='black', verticalalignment='center')

x, y, arrow_length = 0.9, 0.9, 0.1
plt.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords='axes fraction')

# save figure
plt.savefig('maps/1_points.png', dpi=300, bbox_inches='tight')